### Nathan Eskelson

In [137]:
import numpy as np
from scipy.stats import binom

## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

def EuropeanCallDelta(S, X, u, d):
    deltaC = (max((u * S) - X, 0) - max((d * S) - X, 0))/(S * (u - d))
    return deltaC

def EuropeanCallBeta(S, X, u, d, r, T, n):
    betaC = np.exp(-r * (T/n)) * (((u * (max((d * S) - X, 0))) - d * (max((u * S) - X, 0))) / (u - d))
    return betaC

def EuropeanCall(S, X, r, u, d, T, n):
    deltaC = EuropeanCallDelta(S, X, u, d) 
    betaC = EuropeanCallBeta(S, X, u, d, r, T, n)
    callPrice = S * deltaC + betaC
    return callPrice

def EuropeanPutDelta(S, X, u, d):
    deltaP = (max(X - (u * S), 0) - max(X - (d * S), 0))/(S * (u - d))
    return deltaP

def EuropeanPutBeta(S, X, u, d, r, T, n):
    betaP = np.exp(-r * (T/n)) * (((u * (max(X - (d * S), 0))) - d * (max(X - (u * S), 0))) / (u - d))
    return betaP

def EuropeanPut(S, X, r, u, d, T, n):
    deltaP = EuropeanPutDelta(S, X, u, d) 
    betaP = EuropeanPutBeta(S, X, u, d, r, T, n)
    putPrice = S * deltaP + betaP
    return putPrice

def buildTrees(S,X,r,T,v,u,d,n,opt_type):
    spotArr = np.zeros([n + 1, n + 1])
    payoffArr = np.zeros([n + 1, n + 1])
    premArr = np.zeros([n + 1, n + 1])
    deltaArr = np.zeros([n + 1, n + 1])
    betaArr = np.zeros([n + 1, n + 1])
    # build price tree
    for x in range(n + 1):
        for y in range(n + 1):
            if (y - x) >= 0:
                spotArr[x,y] = S * (u ** (y - x)) * (d ** (x))
    # build payoff tree with prem, delta, and beta
    for x in range(n + 1):
        for y in range(n + 1):
            if (y - x) >= 0:
                payoffArr[x,y] = max(spotArr[x,y] - X, 0)
                if opt_type == 'call':
                    premArr[x,y] = EuropeanCall(spotArr[x,y], X, r, u, d, T, n)
                    deltaArr[x,y] = EuropeanCallDelta(spotArr[x,y], X, u, d)
                    betaArr[x,y] = EuropeanCallBeta(spotArr[x,y], X, u, d, r, T, n)
                else:
                    premArr[x,y] = EuropeanPut(spotArr[x,y], X, r, u, d, T, n)
                    deltaArr[x,y] = EuropeanPutDelta(spotArr[x,y], X, u, d)
                    betaArr[x,y] = EuropeanPutBeta(spotArr[x,y], X, u, d, r, T, n)
    
    optionValues = {'price':spotArr, 'payoff':payoffArr, 'premium':premArr,'delta':deltaArr, 'beta':betaArr}
    return optionValues

def americanBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
        
    if verbose:
        print(Ct)
        
    for t in range((n-1), -1, -1):
        for j in range(t + 1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            St[j] = St[j] / u
            np.maximum(Ct[j], payoff(St[j], K))
            print(Ct)
            
    return Ct[0]

def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = False):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    
    ## Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
    
    if verbose:
        print(Ct)
        
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            St[j] = St[j] / u
            Ct[j] = np.maximum(Ct[j], K - St[j])
            print(Ct)
            
    return Ct[0]

def euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, payoff, verbose = False):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    ## Arrays to store the spot prices and option values
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
    
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
    

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
          
    return Ct[0,0]

    
buildTrees(100, 95, .08, .5, 0, 1.3, .8, 2, 'call')

{'price': array([[100., 130., 169.],
        [  0.,  80., 104.],
        [  0.,   0.,  64.]]), 'payoff': array([[ 5., 35., 74.],
        [ 0.,  0.,  9.],
        [ 0.,  0.,  0.]]), 'premium': array([[15.10887429, 36.88112604, 75.88112604],
        [ 0.        ,  3.8851391 , 17.35362133],
        [ 0.        ,  0.        ,  0.        ]]), 'delta': array([[0.7       , 1.        , 1.        ],
        [0.        , 0.225     , 0.77307692],
        [0.        , 0.        , 0.        ]]), 'beta': array([[-54.89112571, -93.11887396, -93.11887396],
        [  0.        , -14.1148609 , -63.04637867],
        [  0.        ,   0.        ,   0.        ]])}

## 1)

In [10]:
def main_1():
    S = 100
    X = 105
    r = 0.08
    T = 0.5
    v = 0.0
    u = 1.3
    d = 0.8
    n = 1
    
    callPrice = EuropeanCall(S, X, r, u, d, T, n)
    print("Call Premium = {0:.4f}".format(callPrice))
    callDelta = EuropeanCallDelta(S, X, u, d)
    print("Call Delta = {0:.4f}".format(callDelta))
    callBeta = EuropeanCallBeta(S, u, d, r, T, n, X)
    print("Call Beta = {0:.4f}".format(callBeta))
    putPrice = EuropeanPut(S, X, r, u, d, T, n)
    print("Put Premium = {0:.4f}".format(putPrice))
    putDelta = EuropeanPutDelta(S, X, u, d)
    print("Put Delta = {0:.4f}".format(putDelta))
    putBeta = EuropeanPutBeta(S, u, d, r, T, n, X)
    print("Put Beta = {0:.4f}".format(putBeta))
    
main_1()

Call Premium = 11.5684
Call Delta = 0.5000
Call Beta = -1.2938
Put Premium = 12.4513
Put Delta = -0.5000
Put Beta = 0.0000


## 2)

In [11]:
def main_2():
    S = 100
    X = 95
    r = 0.08
    T = 0.5
    v = 0.0
    u = 1.3
    d = 0.8
    n = 1
    
    putPrice = EuropeanPut(S, X, r, u, d, T, n)
    print("a. Put Price = {0:.4f}".format(putPrice))
    callPrice = EuropeanCall(S, X, r, u, d, T, n)
    print("b. Call Price = {0:.4f}".format(callPrice)) 
    print("\n Buy 17 shares of the stock and borrow 53.8042 to create a synthetic call. After we will sell the call option and make 17 - 16.1958 which is .8042\n")
    print("c. We want to buy the call option as they have it undervalued. After sell 17 units of a share and lend 53.8042. We then profit 16.1958 - 15 which is 1.1958")
    
main_2()

a. Put Price = 7.4708
b. Call Price = 16.1958

 Buy 17 shares of the stock and borrow 53.8042 to create a synthetic call. After we will sell the call option and make 17 - 16.1958 which is .8042

c. We want to buy the call option as they have it undervalued. After sell 17 units of a share and lend 53.8042. We then profit 16.1958 - 15 which is 1.1958


## 3)

In [30]:
def main_3():
    S = 100
    X = 95
    r = 0.08
    T = 0.5
    v = 0.00
    u = 1.3
    d = 0.8
    n = 2
    
    trees3 = buildTrees(S,X,r,T,v,u,d,n,opt_type = 'call') 
    for key, value in trees3.items():
        print(key,'\n',value,'\n')
main_3()    

price 
 [[100. 130. 169.]
 [  0.  80. 104.]
 [  0.   0.  64.]] 

payoff 
 [[ 5. 35. 74.]
 [ 0.  0.  9.]
 [ 0.  0.  0.]] 

premium 
 [[15.10887429 36.88112604 75.88112604]
 [ 0.          3.8851391  17.35362133]
 [ 0.          0.          0.        ]] 

delta 
 [[0.7        1.         1.        ]
 [0.         0.225      0.77307692]
 [0.         0.         0.        ]] 

beta 
 [[-54.89112571 -93.11887396 -93.11887396]
 [  0.         -14.1148609  -63.04637867]
 [  0.           0.           0.        ]] 



## 4)

In [13]:
def main_4():
    S = [80, 90, 110, 120, 130]
    X = 95
    r = 0.08
    T = 0.5
    v = 0.00
    u = 1.3
    d = 0.8
    n = 2
    
    for i in S:
        print('Delta Trees for Spot price of ${0:.0f}'.format(i))
        trees4 = buildTrees(i,X,r,T,v,u,d,n,opt_type = 'call')
        print(trees4['delta'],'\n')
main_4()    

Delta Trees for Spot price of $80
[[0.225      0.77307692 1.        ]
 [0.         0.         0.31634615]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $90
[[0.48888889 0.97606838 1.        ]
 [0.         0.         0.57008547]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $110
[[0.87272727 1.         1.        ]
 [0.         0.44090909 0.93916084]
 [0.         0.         0.        ]] 

Delta Trees for Spot price of $120
[[1.         1.         1.        ]
 [0.         0.62083333 1.        ]
 [0.         0.         0.12604167]] 

Delta Trees for Spot price of $130
[[1.         1.         1.        ]
 [0.         0.77307692 1.        ]
 [0.         0.         0.31634615]] 



## 5a)

In [161]:
S = 100
X = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1.0
n = 3

callPrcA = euroBinomPricerRecursive(S, X, r, v, q, T, n, callPayoff,  verbose = False)
callPrcE = euroBinomPricerRecursiveMatrix(S, X, r, v, q, T, n, callPayoff,  verbose = False)
putPrcA = euroBinomPricerRecursive(S, X, r, v, q, T, n, putPayoff,  verbose = False)
putPrcE = euroBinomPricerRecursiveMatrix(S, X, r, v, q, T, n, putPayoff,  verbose = False)


print(f"The American call is {callPrcE : 0.3f}")

[56.64406241 33.8147424   0.          0.        ]
[56.64406241 15.04032855  0.          0.        ]
[56.64406241 15.04032855 20.40351726  0.        ]
[33.14931753 15.04032855 20.40351726  0.        ]
[33.14931753 17.48114202 20.40351726  0.        ]
[23.99012609 17.48114202 20.40351726  0.        ]
[ 0.          0.          3.89933488 30.57157332]
[ 0.          2.06235672  3.89933488 30.57157332]
[ 0.          2.06235672 20.40351726 30.57157332]
[ 1.09077967  2.06235672 20.40351726 30.57157332]
[ 1.09077967 11.7087201  20.40351726 30.57157332]
[ 6.67790123 11.7087201  20.40351726 30.57157332]
The American call is  18.283


## 5b)

In [145]:
callPrcE = euroBinomPricerRecursiveMatrix(S, X, r, v, q, T, n, callPayoff,  verbose = False)

print(f"The European Call Premium: {callPrcE : 0.3f}")

The European Call Premium:  18.283


## 5c)

In [147]:
putPrcE = euroBinomPricerRecursiveMatrix(S, X, r, v, q, T, n, putPayoff,  verbose = False)

print(f"The European Put Premium: {putPrcE : 0.3f}")

The European Put Premium:  5.979


## 5d)

In [163]:
print(f"The American put is {putPrcA : 0.3f}")

The American put is  6.678


## 6a)

In [159]:
S = 40
K = 40
r = 0.08
v = 0.30
q = 0.0
T = 0.5
n = 3
h = T/n

u6a = np.exp((r - q) * h + v * np.sqrt(h))
d6a = np.exp((r - q) * h - v * np.sqrt(h))

callPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, callPayoff,  verbose = False)
callPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = False)
putPrcA = euroBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff,  verbose = False)
putPrcE = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff,  verbose = False)

print(f"u = {u6a : 0.3f}")
print(f"d = {d6a : 0.3f}")

[13.01309727  7.05673217  0.          0.        ]
[13.01309727  3.26869392  0.          0.        ]
[13.01309727  3.26869392  7.84406148  0.        ]
[7.73902974 3.26869392 7.84406148 0.        ]
[7.73902974 5.6208478  7.84406148 0.        ]
[6.52754713 5.6208478  7.84406148 0.        ]
[ 0.          0.          3.16660277 11.16886174]
[ 0.          1.65788424  3.16660277 11.16886174]
[ 0.          1.65788424  7.84406148 11.16886174]
[ 0.86799019  1.65788424  7.84406148 11.16886174]
[ 0.86799019  4.8747172   7.84406148 11.16886174]
[ 2.95422756  4.8747172   7.84406148 11.16886174]
u =  1.145
d =  0.897


## 6b)

In [156]:
print(f"The American Call is {callPrcE : 0.3f}")

The American Call is  4.377


In [157]:
print(f"The American Put is {putPrcA : 0.3f}")

The American Put is  2.954


In [154]:
print(f"The European Call Premium: {callPrcE : 0.3f}")

The European Call Premium:  4.377


In [155]:
print(f"The European Put Premium: {putPrcE : 0.3f}")

The European Put Premium:  2.809
